# Group Progject Part 3

Ananov Georgy - 890980  
Le Duong - 894834
Nguyen Tam - 906968
Pham Hieu - 895118

## Library Imports

In [5]:
import psycopg2
from psycopg2 import Error
from sqlalchemy import create_engine, text
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

## Database connection

First things first, we need to connect to the *PostgreSQL* database we designed and implemented during the previous rounds. We can use methods from the `psycopg2` and `sqlalchemy` libraries to establish a connection and execute SQL queries.

In [7]:
# Credentials to connect to Postgres database
database="grp21_vaccinedist"   
user='grp21'       
password='B5!BpWYT' 
host='dbcourse2022.cs.aalto.fi'
port= '5432'

try:
    connection = psycopg2.connect(
                                    database=database,              
                                    user=user,       
                                    password=password,   
                                    host=host,
                                    port=port
                                )
    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")
    DIALECT = 'postgresql+psycopg2://'
    db_uri = "%s:%s@%s/%s" % (user, password, host, database)
    engine = create_engine(DIALECT + db_uri)
    psql_conn  = engine.connect()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

You are connected to -  ('PostgreSQL 12.11 (Ubuntu 12.11-0ubuntu0.20.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0, 64-bit',) 



With the connection established, we can proceed to conducting the analysis tasks.

## Data analysis

### Question 1

Let's start by pulling data about patients and symptoms from the database into a pandas dataframe using an SQL query.

In [8]:
query = '''
    SELECT ssno, gender, birthday, symptom, date
    FROM patient, diagnosed
    WHERE patient = ssno
'''
df = pd.read_sql(query, psql_conn)
df.head()


,ssno,gender,birthday,symptom,date
0,790503-394M,F,1979-05-03,anaphylaxia,2021-01-30
1,000127-4899,F,2000-01-27,blurring of vision,2021-02-01
2,090707-295R,F,2009-07-07,chest pain,2021-02-15
3,810616-9029,M,1981-06-16,chest pain,2021-04-30
4,701127-5340,F,1970-11-27,chills,2021-04-10


Now we can create a table using the dataframe we obtained.

In [9]:
df.to_sql("patientsymptoms", psql_conn, index = True, if_exists = "replace")


TypeError: NDFrame.to_sql() missing 1 required positional argument: 'con'

### Question 2

### Question 3

### Question 4

### Question 5

### Question 6

### Question 7

### Question 8

### Question 9

### Question 10